## Utilisation de LINQ et Z3 pour la résolution de problèmes

### Installation de Z3.Linq

On appelle le package Nuget correspondant. 




In [1]:
#r "nuget: Z3.Linq"

Installed Packages Z3.Linq, 2.0.1

### Qu'est-ce que Z3.Linq ?

**Z3.Linq** est une bibliothèque qui combine deux technologies puissantes:

1. **Z3**: Un solveur SMT (Satisfiability Modulo Theories) développé par Microsoft Research
   - Résout des problèmes de satisfaction de contraintes
   - Supporte les entiers, réels, tableaux, structures de données, etc.

2. **LINQ (Language Integrated Query)**: Une syntaxe C# pour exprimer des requêtes de manière déclarative
   - Permet d'écrire les contraintes Z3 avec la syntaxe familière de LINQ
   - Les expressions lambda sont automatiquement traduites en formules SMT

**Avantage principal**: Au lieu d'écrire du code impératif pour résoudre un problème, on **déclare** les contraintes et Z3 trouve une solution (si elle existe).

**Cas d'usage typiques**:
- Résolution de systèmes d'équations
- Problèmes de planification et d'ordonnancement
- Vérification formelle
- Puzzles logiques (Sudoku, n-Queens, etc.)


### Exemple court

In [2]:
using Z3.Linq; 

using (var ctx = new Z3Context())
      {
        var theorem = from t in ctx.NewTheorem<Symbols<int, int, int, int, int>>()
              where t.X1 - t.X2 >= 1
              where t.X1 - t.X2 <= 3
              where t.X1 == (2 * t.X3) + t.X5
              where t.X3 == t.X5
              where t.X2 == 6 * t.X4
              select t;
var solution = theorem.Solve();
Console.WriteLine("X1 = {0}, X2 = {1}, X3 = {2}, X4 = {3}, X5 = {4}", solution.X1, solution.X2, solution.X3, solution.X4, solution.X5);

}

X1 = 3, X2 = 0, X3 = 1, X4 = 0, X5 = 1


### Analyse de l'exemple

**Ce que fait le code:**

Le code définit un système de 5 contraintes linéaires sur 5 variables entières (X1, X2, X3, X4, X5):

```
X1 - X2 ∈ [1, 3]           (deux contraintes)
X1 = 2·X3 + X5
X3 = X5
X2 = 6·X4
```

**Syntaxe LINQ To Z3:**
- `ctx.NewTheorem<Symbols<int, int, int, int, int>>()`: Crée un théorème avec 5 variables entières
- `where t.X1 - t.X2 >= 1`: Ajoute une contrainte (traduite en formule SMT)
- `theorem.Solve()`: Demande à Z3 de trouver une affectation satisfaisant toutes les contraintes

**Résultat attendu:**

Le solveur Z3 trouve une solution (il peut y en avoir plusieurs):
- Une affectation possible: X1 = 8, X2 = 6, X3 = 2, X4 = 1, X5 = 2

> **Note**: Z3 est un solveur SMT (Satisfiability Modulo Theories) capable de résoudre des contraintes sur des entiers, des réels, des tableaux, etc. LINQ To Z3 offre une interface C# naturelle pour exprimer ces contraintes.


### Le problème des missionnaires et cannibales

**Énoncé classique**: 3 missionnaires et 3 cannibales doivent traverser une rivière à l'aide d'une barque pouvant transporter au maximum 2 personnes. Si à un moment donné, sur l'une des deux rives, les cannibales sont plus nombreux que les missionnaires, ces derniers se font dévorer.

**Objectif**: Trouver une séquence de traversées permettant à tous de passer sains et saufs.

**Modélisation pour Z3.Linq:**
- **Variables**: État de chaque rive à chaque étape (nombre de missionnaires et cannibales)
- **Contraintes**: Capacité de la barque, sécurité des missionnaires, état initial/final
- **Solution**: Séquence d'états satisfaisant toutes les contraintes

La classe suivante encode ce problème de manière déclarative pour être résolu par Z3.


### Classe de missionnaires et cannibales

### Détails de la modélisation

La classe `CanibalsAndMissionaries` encode le problème en trois parties:

#### 1. Variables d'état

| Variable | Type | Signification |
|----------|------|---------------|
| `Missionaries[i]` | `int[]` | Nombre de missionnaires sur la rive de départ à l'étape i |
| `Canibals[i]` | `int[]` | Nombre de cannibales sur la rive de départ à l'étape i |
| `Length` | `int` | Nombre total d'étapes dans le chemin |

> **Note**: La rive d'arrivée se déduit par soustraction: `MissionnairesArrivée = NbMissionaries - Missionaries[i]`

#### 2. Contraintes du problème

**Contraintes globales:**
- État initial: Tous les missionnaires et cannibales sont sur la rive de départ
- État final: Tous sont sur la rive d'arrivée (0 personnes sur la rive de départ)

**Contraintes de transition (modèle d'actions):**
- Étapes paires (i % 2 == 0): La barque traverse de départ vers arrivée → perte de 1 à `SizeBoat` personnes
- Étapes impaires (i % 2 == 1): La barque revient d'arrivée vers départ → gain de 1 à `SizeBoat` personnes

**Contrainte de sécurité (invariant critique):**
```csharp
// Sur chaque rive, jamais plus de cannibales que de missionnaires
// (sauf si aucun missionnaire sur la rive)
(Missionaries[i] == 0 || Missionaries[i] >= Canibals[i])
```

#### 3. Construction du théorème avec LINQ

La méthode `Create()` construit progressivement le théorème en ajoutant des clauses `Where()`:
- Chaque appel à `Where()` ajoute une contrainte SMT
- La boucle `for` génère des contraintes pour chaque étape du chemin
- Z3.Linq traduit automatiquement les expressions lambda en formules logiques

Cette approche **déclarative** contraste avec la programmation impérative classique: on décrit **quoi** trouver, pas **comment** le trouver.


###	Affirmer les contraintes

L’affirmation des contraintes des arbres d’expression LINQ sur la classe d’état et l’étape suivante. On se base pour ça sur la classe Theorem de LINQ To Z3 :


In [3]:
public class CanibalsAndMissionaries
{
    
    // Le nombre de canibales et missionaires (3 dans le problème original)
    public int NbMissionaries { get; set; } = 3;
    // La taille de la barque (2 dans le projet original)
    public int SizeBoat { get; set; } = 2;

    // La longueur du chemin calculé
    private int _length;

    //La propriété qui permet d'accéder à la taille du chemin dans Z3
    public int Length
    {
      get => _length;
      set
      {
        _length = value;
        // Quand la longueur est déterminée par Z3, on initialise les tableaux pour pouvoir récupérer les valeurs
        Canibals = new int[value];
        Missionaries = new int[value];
      }
    }

    // Un tableau contenant à chaque étape le nombre de canibales sur la berge de départ
    public int[] Canibals { get; set; }
    // Un tableau contenant à chaque étape le nombre de missionaires sur la berge de départ
    public int[] Missionaries { get; set; }

    /// <summary>
    /// Une représentation lisible de la solution proposée
    /// </summary>
    /// <returns>une chaine de caractère ou chaque ligne est une étape du chemin</returns>
    public override string ToString()
    {
      var sb = new StringBuilder();
      for (int i = 0; i < Canibals.Length; i++)
      {
        sb.AppendLine($"{i + 1} - (({Missionaries[i]}M, {Canibals[i]}C, {1 - i % 2}), ({(i % 2)}, {NbMissionaries - Missionaries[i]}M, {NbMissionaries - Canibals[i]}C))");
      }

      return sb.ToString();

    }
    
    /// <summary>
    /// La méthode qui permet la création du théorème associé au problème
    /// </summary>
    /// <param name="context">Le contexte Z3 qui devra interpréter les contraintes</param>
    /// <param name="entity">Une valeur du problème servant de modèle pour définir les paramètres principaux</param>
    /// <returns>Un théorème de notre environnement qui peut être filtré et résolu</returns>
    public static Theorem<CanibalsAndMissionaries> Create(Z3Context context, CanibalsAndMissionaries entity)
    {
      // On créée une instance du théorème, sans contraintes, puis on va rajouter les contraintes une à une
      var theorem = context.NewTheorem<CanibalsAndMissionaries>();
      
    // Contraintes globales
      // On récupère les contraintes globales qui seront injectées sous forme de constante dans la lambda expression
      var sizeBoat = entity.SizeBoat;
      int nbMissionaries = entity.NbMissionaries;
      int maxlength = entity.Length;

      theorem = theorem.Where(caM => caM.NbMissionaries == nbMissionaries);
      theorem = theorem.Where(caM => caM.SizeBoat == sizeBoat);
      // Etat initial
        theorem = theorem.Where(caM => caM.Missionaries[0] == caM.NbMissionaries && caM.Canibals[0] == caM.NbMissionaries);

      //Modèle de transition
      // On filtre à chaque étape selon les actions possible
      for (int iclosure = 0; iclosure < maxlength; iclosure++)
      {
        var i = iclosure;
        //Les deux rives contiennent entre 0 et 3 personnes
        theorem = theorem.Where(caM => caM.Canibals[i] >= 0
                                       && caM.Canibals[i] <= caM.NbMissionaries
                                       && caM.Missionaries[i] >= 0
                                       && caM.Missionaries[i] <= caM.NbMissionaries);
        if (i % 2 == 0)
        {
          // Aux itérations paires, la rive de départ perd entre 1 et SizeBoat personnes 
          theorem = theorem.Where(caM => caM.Canibals[i + 1] <= caM.Canibals[i]
                                         && caM.Missionaries[i + 1] <= caM.Missionaries[i]
                                         && caM.Canibals[i + 1] + caM.Missionaries[i + 1] - caM.Canibals[i] - caM.Missionaries[i] < 0
                                         && caM.Canibals[i + 1] + caM.Missionaries[i + 1] - caM.Canibals[i] - caM.Missionaries[i] >= -caM.SizeBoat);
        }
        else
        {
          // Aux itérations impaires, la rive de départ gagne entre 1 et SizeBoat personnes 
          theorem = theorem.Where(caM =>
                                    caM.Canibals[i + 1] >= caM.Canibals[i]
                                    && caM.Missionaries[i + 1] >= caM.Missionaries[i]
                                    && caM.Canibals[i + 1] + caM.Missionaries[i + 1] - caM.Canibals[i] - caM.Missionaries[i] > 0
                                    && caM.Canibals[i + 1] + caM.Missionaries[i + 1] - caM.Canibals[i] - caM.Missionaries[i] <= caM.SizeBoat);

        }

        //Jamais moins de missionnaire que de cannibal sur chacune des rives
        theorem = theorem.Where(caM => (caM.Missionaries[i] == 0 || (caM.Missionaries[i] >= caM.Canibals[i]))
                                 && (caM.Missionaries[i] == caM.NbMissionaries || ((caM.NbMissionaries - caM.Missionaries[i]) >= (caM.NbMissionaries - caM.Canibals[i]))));

      }


        // Test de but
      // A l'arrivée, plus personne sur la rive de départ
      theorem = theorem.Where(
        caM => caM.Length > 0
               && caM.Length < maxlength
               && caM.Missionaries[caM.Length - 1] == 0
               && caM.Canibals[caM.Length - 1] == 0
      );


      return theorem;
    }
    
}

###	Obtenir la solution 
LINQ To Z3 nous donne la solution sous forme d’ un objet POCO (Plain Old CLR Object) du type de paramètre générique T du théorème. 


In [4]:
using System.Diagnostics;
var stopWatch = new Stopwatch();
      stopWatch.Start();
      TimeSpan debutChrono;
    // Solving Canibals & Missionaires
      var can = new CanibalsAndMissionaries(){NbMissionaries = 3, SizeBoat = 2, Length = 30};

      using (var ctx = new Z3Context())
      {
        var theorem = CanibalsAndMissionaries.Create(ctx, can);

        debutChrono = stopWatch.Elapsed;

        //Print(theorem);
        var result = theorem.Solve();

        // affichage du résultat
        display($"Durée Cannibales et Missionaires {stopWatch.Elapsed - debutChrono}");
        display(result);

      }


Durée Cannibales et Missionaires 00:00:00.2379193

NbMissionaries,3
SizeBoat,2
Length,22
Canibals,"[ 3, 2, 3, 2, 3, 2, 2, 2, 2, 0, 1, 0, 2, 0, 1, 1, 2, 2, 3, 1 ... (2 more) ]"
Missionaries,"[ 3, 2, 3, 2, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 1, 2, 0, 0, 0 ... (2 more) ]"


### Interprétation des résultats

Le code ci-dessus résout le problème classique des **3 missionnaires et 3 cannibales** avec une barque de capacité 2.

**Paramètres du problème:**
- `NbMissionaries = 3`: 3 missionnaires et 3 cannibales
- `SizeBoat = 2`: La barque peut transporter au maximum 2 personnes
- `Length = 30`: Limite supérieure du nombre d'étapes à explorer

**Sortie attendue:**
- Un chemin solution avec le nombre d'étapes nécessaires
- Chaque ligne au format: `(MissionnairesDépart, CannibalsDépart, Barque), (Barque, MissionnairesArrivée, CannibalesArrivée)`
- La durée d'exécution du solveur Z3

> **Note**: La méthode `Solve()` trouve **une** solution satisfaisant les contraintes, mais pas nécessairement la plus courte. Pour obtenir la solution optimale, il faut utiliser `Optimize()` (voir section suivante).


### Recherche de la solution la plus courte

In [5]:
using System.Diagnostics;
var stopWatch = new Stopwatch();
      stopWatch.Start();
      TimeSpan debutChrono;
    // Solving Canibals & Missionaires
      var can = new CanibalsAndMissionaries(){NbMissionaries = 3, SizeBoat = 2, Length = 30};

      using (var ctx = new Z3Context())
      {
        var theorem = CanibalsAndMissionaries.Create(ctx, can);

        debutChrono = stopWatch.Elapsed;

        //Print(theorem);
        var result = theorem.Optimize(Optimization.Minimize, objMnC => objMnC.Length);

        // affichage du résultat
        display($"Durée Cannibales et Missionaires {stopWatch.Elapsed - debutChrono}");
        display(result);

      }




Durée Cannibales et Missionaires 00:00:00.1094443

NbMissionaries,3
SizeBoat,2
Length,12
Canibals,"[ 3, 1, 2, 0, 1, 1, 2, 2, 3, 1, 1, 0 ]"
Missionaries,"[ 3, 3, 3, 3, 3, 1, 2, 0, 0, 0, 1, 0 ]"


### Analyse de l'optimisation

**Différence entre `Solve()` et `Optimize()`:**

- **`Solve()`**: Trouve **une** solution satisfaisant toutes les contraintes, sans garantie d'optimalité
- **`Optimize(Optimization.Minimize, objMnC => objMnC.Length)`**: Trouve la solution avec la **plus petite** valeur de `Length`

Dans le cas des missionnaires et cannibales, cela signifie:
- `Solve()` peut retourner un chemin de 15 étapes alors qu'il existe un chemin de 11 étapes
- `Optimize()` garantit de trouver le chemin le plus court possible

> **Performance**: L'optimisation est plus coûteuse en temps de calcul, car le solveur doit explorer l'espace de recherche pour prouver qu'aucune solution plus courte n'existe.

**Résultat attendu**: Le chemin optimal pour 3 missionnaires et 3 cannibales avec une barque de capacité 2 est de **11 étapes**.


In [6]:
using System.Diagnostics;
var stopWatch = new Stopwatch();
      stopWatch.Start();
      TimeSpan debutChrono;
    // Solving Canibals & Missionaires
      var can = new CanibalsAndMissionaries(){NbMissionaries = 30, SizeBoat = 7, Length = 100};

      using (var ctx = new Z3Context())
      {
        var theorem = CanibalsAndMissionaries.Create(ctx, can);

        debutChrono = stopWatch.Elapsed;

        //Print(theorem);
        var result = theorem.Optimize(Optimization.Minimize, objMnC => objMnC.Length);

        // affichage du résultat
        display($"Durée Cannibales et Missionaires {stopWatch.Elapsed - debutChrono}");
        display(result);

      }




Durée Cannibales et Missionaires 00:00:34.8835804

NbMissionaries,30
SizeBoat,7
Length,28
Canibals,"[ 30, 23, 24, 24, 25, 22, 23, 20, 21, 18, 19, 16, 17, 14, 15, 12, 13, 10, 11, 8 ... (8 more) ]"
Missionaries,"[ 30, 30, 30, 24, 25, 22, 23, 20, 21, 18, 19, 16, 17, 14, 15, 12, 13, 10, 11, 8 ... (8 more) ]"


### Conclusion et analyse comparative

Ce notebook a illustré l'utilisation de **Z3.Linq** pour résoudre des problèmes de satisfaction de contraintes en combinant la puissance du solveur SMT Z3 avec la syntaxe LINQ de C#.

#### Comparaison des approches

| Approche | Cas simple (3M, 3C) | Cas complexe (30M, 30C) | Avantages | Limitations |
|----------|-------------------|----------------------|-----------|-------------|
| **Solve()** | Rapide | Rapide | Trouve une solution | Pas forcément optimale |
| **Optimize()** | Très rapide | Plus lent | Solution optimale | Coût de calcul plus élevé |

#### Points clés à retenir

1. **Modélisation déclarative**: Les contraintes sont exprimées naturellement avec LINQ
2. **Abstraction puissante**: Z3.Linq traduit automatiquement les expressions LINQ en formules SMT
3. **Optimisation**: La méthode `Optimize()` permet de minimiser/maximiser des objectifs
4. **Scalabilité**: Le solveur gère des instances de taille significative (30 personnes)

> **Note pratique**: Pour des problèmes de grande taille, il est recommandé de fournir une borne supérieure réaliste pour `Length` afin d'éviter l'explosion combinatoire.

#### Applications possibles

- Planification de tâches
- Problèmes de routage et d'allocation de ressources
- Vérification formelle de propriétés
- Résolution de puzzles logiques (Sudoku, n-Queens, etc.)
